# 用元类来注册子类

In [1]:
import logging
import json

元类还有一个用途：在程序中自动注册类型。对于需要反向查找的场合，很有用的。可以在简单的标识符与对应的类之间，建立映射关系。

**示例：**将Python对象表示为JSON格式的序列化数据。定义一个基类，可以记录程序调用本类构造器时所用的参数，并将其转换为JSON字典。

In [2]:
class Serializable(object):
    def __init__(self, *args):
        self.args = args

    def serialize(self):
        return json.dumps({'args': self.args})

In [3]:
class Point2D(Serializable):
    def __init__(self, x, y):
        super().__init__(x, y)
        self.x = x
        self.y = y

    def __repr__(self):
        return 'Point2D(%d, %d)' % (self.x, self.y)

In [4]:
point = Point2D(5, 3)
print('Object:    ', point)
print('Serialized:', point.serialize())

Object:     Point2D(5, 3)
Serialized: {"args": [5, 3]}


对JSON字符串执行反序列化操作，并构建出该字符串所表示的Point2D对象。

In [5]:
class Deserializable(Serializable):
    @classmethod
    def deserialize(cls, json_data):
        params = json.loads(json_data)
        return cls(*params['args'])

有了Deserializable，有了一种通用的方式，对简单且不可变的对象执行序列化和反序列化操作。

In [6]:
class BetterPoint2D(Deserializable):
    def __init__(self, x, y):
        super().__init__(x, y)
        self.x = x
        self.y = y

    def __repr__(self):
        return 'BetterPoint2D(%d, %d)' % (self.x, self.y)

In [7]:
point = BetterPoint2D(5, 3)
print('Before:    ', point)
data = point.serialize()
print('Serialized:', data)
after = BetterPoint2D.deserialize(data)
print('After:     ', after)

Before:     BetterPoint2D(5, 3)
Serialized: {"args": [5, 3]}
After:      BetterPoint2D(5, 3)


**缺点：**需要提前知道序列化的数据是什么类型，然后才能对其做反序列化操作。

**理想的方案：**有很多类都可以把本类对象转换为JSON格式的序列化字符串，但是只需要一个公共的反序列化函数，就可以将任意的JSON字符串还原成相应的Python对象。

## 第一次改进：把序列化对象的类名写到JSON数据里面

In [8]:
class BetterSerializable(object):
    def __init__(self, *args):
        self.args = args

    def serialize(self):
        return json.dumps({
            'class': self.__class__.__name__,
            'args': self.args,
        })

    def __repr__(self):
        return '%s(%s)' % (
            self.__class__.__name__,
            ', '.join(str(x) for x in self.args))

In [9]:
registry = {}

In [10]:
# 把将来可能执行反序列化操作的类，都注册一遍。
def register_class(target_class):
    registry[target_class.__name__] = target_class

In [11]:
def deserialize(data):
    params = json.loads(data)
    name = params['class']
    target_class = registry[name]
    return target_class(*params['args'])

In [12]:
class EvenBetterPoint2D(BetterSerializable):
    def __init__(self, x, y):
        super().__init__(x, y)
        self.x = x
        self.y = y

In [13]:
register_class(EvenBetterPoint2D)

In [14]:
point = EvenBetterPoint2D(5, 3)
print('Before:    ', point)
data = point.serialize()
print('Serialized:', data)
after = deserialize(data)
print('After:     ', after)

Before:     EvenBetterPoint2D(5, 3)
Serialized: {"class": "EvenBetterPoint2D", "args": [5, 3]}
After:      EvenBetterPoint2D(5, 3)


**缺点：**开发者可能会忘记调用register_class函数。

In [15]:
class Point3D(BetterSerializable):
    def __init__(self, x, y, z):
        super().__init__(x, y, z)
        self.x = x
        self.y = y
        self.z = z

In [16]:
# Forgot to call register_class! Whoops!
try:
    point = Point3D(5, 9, -4)
    data = point.serialize()
    deserialize(data)
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "<ipython-input-16-2deb01ec07bb>", line 5, in <module>
    deserialize(data)
  File "<ipython-input-11-4db209d626fc>", line 4, in deserialize
    target_class = registry[name]
KeyError: 'Point3D'


## 第二次改进：在继承BetterSerializable，程序会自动调用register_class函数，将新的子类注册好。

In [17]:
class Meta(type):
    def __new__(meta, name, bases, class_dict):
        cls = type.__new__(meta, name, bases, class_dict)
        register_class(cls)
        return cls

In [18]:
class RegisteredSerializable(BetterSerializable, metaclass=Meta):
    pass

In [19]:
class Vector3D(RegisteredSerializable):
    def __init__(self, x, y, z):
        super().__init__(x, y, z)
        self.x, self.y, self.z = x, y, z

In [20]:
v3 = Vector3D(10, -7, 3)
print('Before:    ', v3)
data = v3.serialize()
print('Serialized:', data)
print('After:     ', deserialize(data))

Before:     Vector3D(10, -7, 3)
Serialized: {"class": "Vector3D", "args": [10, -7, 3]}
After:      Vector3D(10, -7, 3)
